In [1]:
import pandas as pd
import numpy as np
import os
import requests; from urllib.parse import urlparse
from tqdm.notebook import tqdm

from folium.features import DivIcon
import folium
import pyproj
from pyproj import Proj, transform

import warnings
warnings.filterwarnings(action='ignore')


def katec_to_wgs84(x, y):
    inProj  = Proj('+proj=tmerc +lat_0=38 +lon_0=128 +k=0.9999 +x_0=400000 +y_0=600000 +ellps=bessel +units=m +no_defs +towgs84=-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43')
    outProj = Proj({ 'proj':'latlong', 'datum':'WGS84', 'ellps':'WGS84' })
    return transform( inProj, outProj, x, y )


def address_to_latlon(lng,lat):
    url = f"https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?&x={lng}&y={lat}&input_coord=WGS84"
    result = requests.get(urlparse(url).geturl(),
                          headers={"Authorization":"KakaoAK 5a53ab667089a2a92726783ed5d4deb3"})
    json_obj = result.json()
    if json_obj['meta']['total_count'] > 0:
        for document in json_obj['documents']:
            val = document['region_2depth_name']
    else:
        val = 'na'
    return val

In [2]:
path = f"./data/500/"
file_list = os.listdir(path)
f_list = [file for file in file_list if file.endswith(".csv")]
print(f_list)
os.chdir(path)

['CV500_SEOUL_201912.csv', 'CV500_SEOUL_202001.csv', 'CV500_SEOUL_202002.csv', 'CV500_SEOUL_202003.csv', 'CV500_SEOUL_202004.csv', 'CV500_SEOUL_202005.csv', 'CV500_SEOUL_202006.csv', 'CV500_SEOUL_202007.csv', 'CV500_SEOUL_202008.csv', 'CV500_SEOUL_202009.csv', 'CV500_SEOUL_202010.csv', 'CV500_SEOUL_202011.csv']


In [6]:
for i in tqdm(range(len(f_list))):
    print(f_list[i])
    celldf = pd.read_csv(f_list[i],encoding='UTF-8')   
    p = celldf[['X','Y']].drop_duplicates()
    p['ADMI_CD'] = list(map(lambda x :address_to_latlon(x[0],x[1]) ,list(map(lambda x,y : katec_to_wgs84(x,y) ,p.X,p.Y))))
    celldf = celldf.merge(p,how='left',on=['X','Y'])
    celldf.to_csv(f'./N_'+f_list[i],encoding = "euc-kr" ,index = False)

  0%|          | 0/12 [00:00<?, ?it/s]

CV500_SEOUL_201912.csv
CV500_SEOUL_202001.csv
CV500_SEOUL_202002.csv
CV500_SEOUL_202003.csv
CV500_SEOUL_202004.csv
CV500_SEOUL_202005.csv
CV500_SEOUL_202006.csv
CV500_SEOUL_202007.csv
CV500_SEOUL_202008.csv
CV500_SEOUL_202009.csv
CV500_SEOUL_202010.csv
CV500_SEOUL_202011.csv
